# Бенчмарк: Детекция сарказма (Sarcasm Detection)

**Задача:** Классификация заголовков новостей (Сарказм / Реальность).
**Цель:** Сравнить эффективность классического Fine-Tuning (ModernBERT) с методами автоматической оптимизации промптов (DSPy) на современной LLM.

**Сравниваемые методы:**
1.  **Fine-Tuning:** ModernBERT-large.
2.  **Manual Prompting:** Ручной промпт + Few-Shot.
3.  **DSPy MIPROv2:** Автоматический подбор инструкций и примеров.
4.  **DSPy GEPA:** Генетическая эволюционная оптимизация промпта.

## 1. Подготовка данных

Используем датасет **News Headlines Dataset for Sarcasm Detection**.

Разбиение: Train (200 примеров для оптимизации/обучения) / Test (800 примеров для валидации).

*Малый размер train-сета выбран намеренно для симуляции проблемы ограниченности качественно размеченных данных, что часто встречается в реальной жизни, а также для проверки эффективности Few-Shot методов.*

In [ ]:
import re
from typing import Literal

import pandas as pd
from tqdm import tqdm

import kagglehub
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import GEPA

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

In [34]:
path = kagglehub.dataset_download("rmisra/news-headlines-dataset-for-sarcasm-detection")
data1 = pd.read_json(f'{path}/Sarcasm_Headlines_Dataset_v2.json', lines=True)
data2 = pd.read_json(f'{path}/Sarcasm_Headlines_Dataset.json', lines=True)
data1 = data1[['headline', 'is_sarcastic']]
data2 = data2[['headline', 'is_sarcastic']]
data = pd.concat([data2, data2])
data.drop_duplicates(inplace=True)
data.sample(10, random_state=42)

,headline,is_sarcastic
12942,departing employee not quite important enough for send-off,1
12440,college student still managing to look like asshole in picture of ...,1
196,fun sticker placed on child's ventilator,1
24348,powerball officials remove plastic balls from pig urine brine,1
15430,the roots to premiere 2 children's series on amazon,0
8355,reuters journalist leaves iraq after being threatened over story,0
23777,poll shows many americans agree with hillary clinton that women an...,0
22529,congress debates coolness of rush,1
17940,god gets celtic cross tattooed on back,1
2158,halloween decorations blending in nicely with christmas lights,1


In [35]:
data = data.sample(1000, random_state=42).reset_index(drop=True)

In [ ]:
data_train, data_test = train_test_split(
    data,
    test_size=0.8,
    stratify=data.is_sarcastic,
    random_state=42,
)

data_train = data_train.rename(columns={"is_sarcastic": "labels"})
data_test = data_test.rename(columns={"is_sarcastic": "labels"})

len(data_train), len(data_test)

(200, 800)

In [37]:
trainset = [
    dspy.Example(
        headline=row["headline"],
        label=str(row["labels"]),
    ).with_inputs("headline")
    for _, row in data_train.iterrows()
]

testset = [
    dspy.Example(
        headline=row["headline"],
        label=str(row["labels"]),
    ).with_inputs("headline")
    for _, row in data_test.iterrows()
]

len(trainset), len(testset)

(200, 800)

## 2.Supervised Fine-Tuning (BERT-like)

Обучаем несколько BERT-подобных моделей на стандартных параметрах.

In [38]:
train_df, val_df = train_test_split(
    data_train,
    test_size=0.2,
    stratify=data_train.labels,
    random_state=42,
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds = Dataset.from_pandas(val_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(data_test.reset_index(drop=True))

len(train_df), len(val_df), len(data_test)

(160, 40, 800)

In [39]:
model_name = "answerdotai/ModernBERT-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["headline"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

train_ds_tok = train_ds.map(tokenize, batched=True)
val_ds_tok   = val_ds.map(tokenize, batched=True)
test_ds_tok  = test_ds.map(tokenize, batched=True)

train_ds_tok.set_format("torch")
val_ds_tok.set_format("torch")
test_ds_tok.set_format("torch")

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [41]:
num_labels = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
    }

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)

training_args = TrainingArguments(
    output_dir="sarcasm-bert",
    learning_rate=2e-5,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tok,
    eval_dataset=val_ds_tok,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.692000,0.572687,0.650000,0.611111
2,0.201800,1.030189,0.725000,0.620690
3,0.003900,0.899999,0.800000,0.764706
4,0.000000,0.996525,0.775000,0.727273
5,0.000000,1.002173,0.775000,0.727273


TrainOutput(global_step=200, training_loss=0.17956246784917312, metrics={'train_runtime': 54.6522, 'train_samples_per_second': 14.638, 'train_steps_per_second': 3.66, 'total_flos': 846044808806400.0, 'train_loss': 0.17956246784917312, 'epoch': 5.0})

In [43]:
test_preds = trainer.predict(test_ds_tok)

logits = test_preds.predictions
labels = test_preds.label_ids
preds = logits.argmax(axis=-1)

print("BERT test accuracy:", accuracy_score(labels, preds))
print("BERT test f1:", f1_score(labels, preds))

BERT test accuracy: 0.75875
BERT test f1: 0.7359781121751026


Модель | Test Acc | Test F1
-------|----------|--------
ModernBERT-large | 0.75875 | 0.7360
Roberta-large | 0.7588 | 0.7215
ModernBERT-base | 0.74625 | 0.7019

## 3. Настройка окружения LLM (vLLM + DSPy)

Запуск сервера инференса **vLLM** в Docker контейнере.

Используемая модель: `unsloth/gemma-3-27b-it-bnb-4bit`.

In [8]:
!docker run -d \
  --gpus all \
  --ipc=host \
  -p 8000:8000 \
  -e PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True \
  -v ~/.cache/huggingface:/root/.cache/huggingface \
  vllm/vllm-openai:latest \
  --model unsloth/gemma-3-27b-it-bnb-4bit \
  --gpu-memory-utilization 0.92 \
  --max-model-len 16384 \
  --enforce-eager

50d017150498d9b4dbca7aef2e3e832dd21de92575b087b51a4a35a4a9239706


In [9]:
!docker logs -f 50d017150498d9b4dbca7aef2e3e832dd21de92575b087b51a4a35a4a9239706

WARNING 12-12 01:45:33 [argparse_utils.py:195] With `vllm serve`, you should provide the model as a positional argument or in a config file instead of via the `--model` option. The `--model` option will be removed in v0.13.
(APIServer pid=1) INFO 12-12 01:45:33 [api_server.py:1772] vLLM API server version 0.12.0
(APIServer pid=1) INFO 12-12 01:45:33 [utils.py:253] non-default args: {'model_tag': 'unsloth/gemma-3-27b-it-bnb-4bit', 'model': 'unsloth/gemma-3-27b-it-bnb-4bit', 'max_model_len': 16384, 'enforce_eager': True, 'gpu_memory_utilization': 0.92}
(APIServer pid=1) INFO 12-12 01:45:37 [model.py:637] Resolved architecture: Gemma3ForConditionalGeneration
(APIServer pid=1) INFO 12-12 01:45:37 [model.py:1750] Using max model len 16384
(APIServer pid=1) INFO 12-12 01:45:38 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=2048.
(APIServer pid=1) WARNING 12-12 01:45:38 [vllm.py:601] Enforce eager set, overriding optimization level to -O0
(APIServer pid=1) INFO 12-1

In [16]:
lm = dspy.LM(
    model="hosted_vllm/unsloth/gemma-3-27b-it-bnb-4bit",
    model_type="chat",
    api_base="http://localhost:8000/v1",
    temperature=0,
)

reflection_lm = dspy.LM(
    model="openrouter/google/gemini-2.5-flash",
    api_key="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",
    api_base="https://openrouter.ai/api/v1",
    temperature=1,
)

dspy.configure(lm=lm)

lm('Hi!')

["Hi there! 👋 \n\nHow can I help you today? Just let me know what you're thinking, whether it's:\n\n* **Asking a question:** About anything! (Within ethical and safe guidelines, of course)\n* **Brainstorming ideas:** For a project, story, or just something fun.\n* **Getting information:** I can try to find information on a topic.\n* **Writing something:** A poem, a story, an email...\n* **Just chatting:** If you're bored and want to talk!\n\nI'm here to assist in any way I can. 😊\n\n\n\n"]

## 4. Manual Prompt Engineering

Ручное написание промпта с использованием техники **CoT** (Chain-of-Thought, неявный) и фиксированных **Few-Shot** примеров.

Цель: установить планку качества, которую может достичь инженер без автоматизации.

In [17]:
MANUAL_PROMPT = """Ты — эксперт-лингвист и специалист по анализу медиа, специализирующийся на классификации текстов и выявлении сатиры. Твоя задача — классифицировать список новостных заголовков на две категории.

### Описание классов:
*   **0 (Real News / Реальные новости):**
    *   Заголовки, описывающие реальные события, политику, экономику, происшествия или новости шоу-бизнеса.
    *   Сюда также относятся статьи формата "lifestyle" (советы, рецепты, мода), даже если они звучат немного банально.
    *   Тон: Информативный, нейтральный или мнимо-сенсационный (но описывающий реальность).
    *   *Ключевые маркеры:* Упоминание реальных дат, статистики, официальных заявлений, стандартные конструкции ("Senate votes...", "Tips for...", "Police report...").

*   **1 (Satire / Сатира):**
    *   Заголовки вымышленных, юмористических новостей (в стиле *The Onion*).
    *   Содержат элементы абсурда, логические несостыковки, иронию или гиперболизацию.
    *   Часто используют обобщенные персонажи ("Area Man", "Local Woman") в нелепых ситуациях или приписывают известным людям (Бог, Папа Римский, президенты) смешные мысли/действия.
    *   *Ключевые маркеры:* Неожиданные повороты в конце предложения, антропоморфизм животных/предметов, явная глупость персонажей, социальный комментарий через абсурд.

### Примеры для обучения (Few-Shot Learning):

**Пример 1 (Реальная новость):**
Headline: "martin o'malley backs $15 national minimum wage"
Label: 0
*Обоснование: Реальный политик высказывается по реальному экономическому вопросу.*

**Пример 2 (Сатира):**
Headline: "fbi director wishes he had some alien thing to cover up"
Label: 1
*Обоснование: Абсурдное желание должностного лица, высмеивающее теории заговора.*

**Пример 3 (Реальная новость - Lifestyle):**
Headline: "8 weird sleep positions couples know all too well"
Label: 0
*Обоснование: Типичный заголовок развлекательной статьи, не содержащий вымысла или сатиры.*

**Пример 4 (Сатира):**
Headline: "area child disappointed to learn parents' love unconditional"
Label: 1
*Обоснование: Использование тропа "Area [Person]" и переворот ожиданий (безусловная любовь обычно радует).*

**Пример 5 (Сатира):**
Headline: "sources: barista not actually flirting with you"
Label: 1
*Обоснование: Юмористическое "разоблачение" бытовой ситуации.*

---

### Твоя задача:
Проанализируй предоставленные пользователем заголовки и присвой им метку (0 или 1).
В ответе верни только цифру:
`Label`

Не пиши никаких пояснений, только результат.

### Входные данные для обработки:
\"{}\""""

In [ ]:
y_true_manual = []
y_pred_manual = []

for ex in tqdm(testset):
    prompt_input = MANUAL_PROMPT.format(ex.headline)
    
    response = lm(prompt_input)
    output = response[0] if isinstance(response, list) else response

    y_true_manual.append(int(ex.label))
    
    found_labels = re.findall(r'[01]', output)
    if found_labels:
        y_pred_manual.append(int(found_labels[-1]))
    else:
        y_pred_manual.append(0)

acc_manual = accuracy_score(y_true_manual, y_pred_manual)
f1_manual = f1_score(y_true_manual, y_pred_manual)

print(f"Manual Prompt Accuracy: {acc_manual * 100:.2f}%")
print(f"Manual Prompt F1: {f1_manual * 100:.2f}%")

100%|██████████| 800/800 [02:56<00:00,  4.53it/s]

Manual Prompt Accuracy: 81.25%
Manual Prompt F1: 82.56%


## 5. Автоматизированный подход DSPy - MIPROv2 (Multi-prompt Instruction PRoposal Optimizer Version 2)

Метод оптимизации, который:
1.  Генерирует вариации инструкций.
2.  Использует Bootstrapping для создания "синтетических" Few-Shot примеров из обучающей выборки.
3.  Оптимизирует комбинацию "Инструкция + Примеры" по метрике Accuracy.

In [19]:
class SarcasmTypedSignature(dspy.Signature):
    """Classify the news headline as sarcastic/fake (1) or not (0)."""
    headline: str = dspy.InputField(desc="The news headline text")
    confidence: float = dspy.OutputField(desc="A score between 0.0 and 1.0 indicating certainty")
    label: Literal["0", "1"] = dspy.OutputField(desc="1 if the title is sarcastic, 0 if not.")

class SarcasmTypedClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classify = dspy.Predict(SarcasmTypedSignature)

    def forward(self, headline: str):
        return self.classify(headline=headline)

In [20]:
program_typed = SarcasmTypedClassifier()

In [21]:
def sarcasm_typed_metric(example, pred, trace=None):
    gold = str(example.label)
    predicted_label = pred.label
    score = 1.0 if predicted_label == gold else 0.0
    return score

In [ ]:
optimizer = dspy.MIPROv2(
    auto=None,
    metric=sarcasm_typed_metric,
    prompt_model=reflection_lm,
    task_model=lm,
    num_candidates=50,
    init_temperature=1,
    verbose=True,
    num_threads=4
)

compiled_classifier = optimizer.compile(
    student=program_typed,
    trainset=trainset,
    
    num_trials=300,
    
    minibatch_size=50,
    minibatch_full_eval_steps=10,
    
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    
    requires_permission_to_run=False
)

compiled_classifier.save("sarcasm_miprov2_optimized.json")

2025/12/12 12:51:21 WARNING dspy.teleprompt.mipro_optimizer_v2: 'requires_permission_to_run' is deprecated and will be removed in a future version.
2025/12/12 12:51:21 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/12/12 12:51:21 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/12/12 12:51:21 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=50 sets of demonstrations...


Bootstrapping set 1/50
Bootstrapping set 2/50
Bootstrapping set 3/50


 22%|██▎       | 9/40 [00:09<00:33,  1.09s/it]


Bootstrapped 8 full traces after 9 examples for up to 1 rounds, amounting to 9 attempts.
Bootstrapping set 4/50


 10%|█         | 4/40 [00:04<00:40,  1.11s/it]


...

...


Predictor 0
i: Examine the given news headline. If it uses humor, exaggeration, or absurdity to deliver a sardonic or satirical message, classify it as sarcastic/fake (1). Otherwise, if it presents information in a straightforward, serious, or factual manner, classify it as not sarcastic (0). Assign a confidence score between 0.0 and 1.0 reflecting your certainty in the classification.
p: Label:


Average Metric: 46.00 / 50 (92.0%): 100%|██████████| 50/50 [00:47<00:00,  1.05it/s]

2025/12/12 17:23:09 INFO dspy.evaluate.evaluate: Average Metric: 46.0 / 50 (92.0%)
2025/12/12 17:23:09 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 92.0 on minibatch of size 50 with parameters ['Predictor 0: Instruction 18', 'Predictor 0: Few-Shot Set 19'].
2025/12/12 17:23:09 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [82.0, 78.0, 72.0, 76.0, 78.0, 78.0, 84.0, 82.0, 88.0, 80.0, 80.0, 84.0, 78.0, 80.0, 88.0, 84.0, 78.0, 78.0, 82.0, 84.0, 88.0, 84.0, 86.0, 92.0, 86.0, 72.0, 80.0, 72.0, 82.0, 80.0, 82.0, 88.0, 88.0, 90.0, 84.0, 64.0, 82.0, 82.0, 86.0, 84.0, 72.0, 88.0, 80.0, 82.0, 88.0, 82.0, 82.0, 78.0, 80.0, 90.0, 84.0, 76.0, 78.0, 86.0, 80.0, 84.0, 94.0, 76.0, 86.0, 88.0, 84.0, 86.0, 84.0, 82.0, 84.0, 80.0, 82.0, 86.0, 82.0, 74.0, 92.0, 92.0, 82.0, 82.0, 82.0, 82.0, 82.0, 88.0, 84.0, 88.0, 82.0, 82.0, 80.0, 88.0, 84.0, 82.0, 78.0, 80.0, 74.0, 82.0, 84.0, 92.0, 80.0, 76.0, 82.0, 88.0, 82.0, 78.0, 80.0, 82.0, 86.0, 82.0, 82.0, 80.0, 78.0, 62.0, 84.0, 88.0, 78.0


Average Metric: 138.00 / 160 (86.2%): 100%|██████████| 160/160 [01:43<00:00,  1.55it/s]

2025/12/12 17:24:52 INFO dspy.evaluate.evaluate: Average Metric: 138.0 / 160 (86.2%)
2025/12/12 17:24:52 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [68.75, 85.0, 83.12, 85.62, 86.25, 83.12, 86.25, 85.62, 84.38, 84.38, 85.62, 85.62, 81.88, 82.5, 88.12, 86.25, 85.62, 84.38, 86.88, 85.62, 84.38, 86.25, 86.25, 87.5, 88.75, 87.5, 81.88, 88.12, 84.38, 87.5, 86.25]
2025/12/12 17:24:52 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 88.75
2025/12/12 17:24:52 INFO dspy.teleprompt.mipro_optimizer_v2: ========================
2025/12/12 17:24:52 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/12/12 17:24:52 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 88.75!


In [ ]:
evaluator = Evaluate(
    devset=trainset,
    metric=sarcasm_typed_metric,
    num_threads=4,
    display_progress=True,
    display_table=True
)

final_score = evaluator(compiled_classifier)

Average Metric: 179.00 / 200 (89.5%): 100%|██████████| 200/200 [00:00<00:00, 8379.39it/s]

2025/12/12 20:47:47 INFO dspy.evaluate.evaluate: Average Metric: 179.0 / 200 (89.5%)


,headline,example_label,confidence,pred_label,sarcasm_typed_metric
0,behind the scenes of an intricate fbi sting,0,0.80,0,✔️ [1.000]
1,"paying organ donors for travel, recovery could enable more low-inc...",0,0.95,0,✔️ [1.000]
2,housekeeper too busy to be sassy,1,0.80,1,✔️ [1.000]
3,maxim skimmed,1,0.90,1,✔️ [1.000]
4,martin o'malley backs $15 national minimum wage,0,0.98,0,✔️ [1.000]
...,...,...,...,...,...
195,wednesday's morning email: latest missile launch from north korea ...,0,0.98,0,✔️ [1.000]
196,the roots to premiere 2 children's series on amazon,0,0.98,0,✔️ [1.000]
197,the wind and sea estate is a 5 star getaway: foraging with friends...,0,0.65,0,✔️ [1.000]
198,"area man to ask his doctor about xenical, propecia, claritin, paxi...",1,0.90,1,✔️ [1.000]


In [25]:
y_true = []
y_pred = []

for ex in tqdm(testset):
    pred = compiled_classifier(headline=ex.headline)
    y_true.append(int(ex.label))
    try:
        y_pred.append(int(pred.label))
    except (ValueError, TypeError):
        y_pred.append(0)

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"MIPROv2 Test Accuracy: {acc * 100:.2f}%")
print(f"MIPROv2 Test F1: {f1 * 100:.2f}%")

100%|██████████| 800/800 [13:26<00:00,  1.01s/it]

MIPROv2 Test Accuracy: 85.25%
MIPROv2 Test F1: 84.99%


## 6. Автоматизированный подход DSPy - GEPA (Genetic-Pareto)

Метод, использующий эволюционные алгоритмы для мутации текста промпта с целью максимизации метрики.

In [26]:
program_typed = SarcasmTypedClassifier()

In [27]:
def sarcasm_gepa_metric(example, pred, trace=None, pred_name=None, pred_trace=None):
    gold = str(example.label)
    pred_label = getattr(pred, 'label', None)
    score = 1.0 if pred_label == gold else 0.0
    return score

In [ ]:
gepa_optimizer = GEPA(
    metric=sarcasm_gepa_metric,
    reflection_lm=reflection_lm,
    
    candidate_selection_strategy="pareto",
    max_metric_calls=5000,
    
    reflection_minibatch_size=50,
    use_merge=True,
    max_merge_invocations=25,
    
    num_threads=4,
    track_stats=True,
    track_best_outputs=True
)

gepa_program = gepa_optimizer.compile(
    student=program_typed,
    trainset=trainset,
    valset=trainset 
)

gepa_program.save("sarcasm_gepa_optimized.json")

2025/12/12 17:38:56 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 5000 metric calls of the program. This amounts to 12.50 full evals on the train+val set.
2025/12/12 17:38:56 INFO dspy.teleprompt.gepa.gepa: Using 200 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
GEPA Optimization:   0%|          | 0/5000 [00:00<?, ?rollouts/s]2025/12/12 17:39:33 INFO dspy.evaluate.evaluate: Average Metric: 146.0 / 200 (73.0%)
2025/12/12 17:39:33 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.73 over 200 / 200 examples
GEPA Optimization:   4%|▍         | 200/5000 [00:37<14:49,  5.40rollouts/s]2025/12/12 17:39:33 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.

Average Metric: 36.00 / 50 (72.0%): 100%|██████████| 50/50 [00:00<00:00, 496.19it/s]

2025/12/12 17:39:33 INFO dspy.evaluate.evaluate: Average Metric: 36.0 / 50 (72.0%)


2025/12/12 17:39:40 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for classify: Classify the news headline as sarcastic/fake (1) or not (0). Consider whether the headline is ironic, uses hyperbole, or describes an absurd or extremely unlikely event. Headlines that sound like they could be from The Onion or similar satirical news outlets should be classified as sarcastic/fake (1). Real news headlines, even if dramatic or discussing unusual real-world events, should be classified as not sarcastic/fake (0).

Here are some examples to guide your classification:
- **Sarcastic/Fake (1):**
    - "montana dems nominate a banjo player for special election -- and he might actually win" (This sounds like a satirical exaggeration of a political situation, implying an unserious candidate could unexpectedly succeed.)
    - "grandmother talking big game about being alive next year" (This is a comical, overly dramatic statement for a personal topic.)
    - "john kasich is seemingly ba

Average Metric: 36.00 / 50 (72.0%): 100%|██████████| 50/50 [00:00<00:00, 5797.88it/s]

2025/12/12 17:40:30 INFO dspy.evaluate.evaluate: Average Metric: 36.0 / 50 (72.0%)


2025/12/12 18:30:00 INFO dspy.teleprompt.gepa.gepa: Iteration 23: Proposed new text for classify: Given a news headline, classify it as sarcastic/fake (1) or not sarcastic/fake (0). Sarcastic/fake headlines often employ hyperbole, absurd scenarios, or unexpected twists on common phrases or news tropes. They may also invent fictional entities or situations that sound plausible but are ultimately ridiculous. Key indicators of sarcasm/fakery include intentional humor, irony, or an exaggerated reality. For example, headlines describing overly dramatic reactions to mundane situations ("area man unable to believe the savings"), personification of animals with human-like, often dark, thoughts ("michelle obama can still hear their little labored breaths when she closes her eyes" referring to pigs, "burmese python shocked at amount of stress man holding in his neck"), or outlandish events related to well-known figures ("pope francis admits god really starting to look old", "biden co-presents be

...

2025/12/12 18:30:49 INFO dspy.evaluate.evaluate: Average Metric: 39.0 / 50 (78.0%)


2025/12/12 18:31:07 INFO dspy.teleprompt.gepa.gepa: Iteration 24: Proposed new text for classify: You are an expert in classifying news headlines as sarcastic/fake or not.
- A label of 1 means the headline is sarcastic/fake.
- A label of 0 means the headline is not sarcastic/fake.

When classifying, pay close attention to headlines that use humor, exaggeration, or irony to convey a message or make a satirical point. Also, look for headlines that present a highly improbable or absurd scenario, as these are often indicators of sarcasm or fakeness. Headlines that are straightforward news reports, even if the news itself is unusual or negative, should generally be classified as not sarcastic/fake.

Here are some specific types of sarcastic or fake headlines to watch out for:
1.  **Headlines making absurd claims or presenting highly improbable scenarios:** These headlines often stretch reality to a comical or unbelievable degree.
    *   "report: music industry made $18 in 2009" (exaggerati

In [ ]:
evaluator_gepa = Evaluate(
    devset=trainset,
    metric=sarcasm_typed_metric,
    num_threads=4,
    display_progress=True,
    display_table=True
)

gepa_train_score = evaluator_gepa(gepa_program)

Average Metric: 162.00 / 200 (81.0%): 100%|██████████| 200/200 [00:00<00:00, 461.17it/s]


2025/12/12 20:47:08 INFO dspy.evaluate.evaluate: Average Metric: 162.0 / 200 (81.0%)


,headline,example_label,confidence,pred_label,sarcasm_typed_metric
0,behind the scenes of an intricate fbi sting,0,0.65,0,✔️ [1.000]
1,"paying organ donors for travel, recovery could enable more low-inc...",0,0.95,0,✔️ [1.000]
2,housekeeper too busy to be sassy,1,0.85,1,✔️ [1.000]
3,maxim skimmed,1,0.95,1,✔️ [1.000]
4,martin o'malley backs $15 national minimum wage,0,0.95,0,✔️ [1.000]
...,...,...,...,...,...
195,wednesday's morning email: latest missile launch from north korea ...,0,0.95,0,✔️ [1.000]
196,the roots to premiere 2 children's series on amazon,0,0.95,0,✔️ [1.000]
197,the wind and sea estate is a 5 star getaway: foraging with friends...,0,0.95,0,✔️ [1.000]
198,"area man to ask his doctor about xenical, propecia, claritin, paxi...",1,0.95,1,✔️ [1.000]


In [30]:
y_true = []
y_pred = []

for ex in tqdm(testset):
    pred = gepa_program(headline=ex.headline)
    y_true.append(int(ex.label))
    try:
        y_pred.append(int(pred.label))
    except (ValueError, TypeError):
        y_pred.append(0)

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"GEPA Test Accuracy: {acc * 100:.2f}%")
print(f"GEPA Test F1: {f1 * 100:.2f}%")

100%|██████████| 800/800 [13:45<00:00,  1.03s/it]

GEPA Test Accuracy: 76.12%
GEPA Test F1: 79.31%


## 7. Итоговые результаты и выводы

Сводная таблица эффективности методов:

| Метод | Модель | Accuracy | F1 Score | Комментарий |
| :--- | :--- | :--- | :--- | :--- |
| **MIPROv2** | Gemma-3-27B (DSPy) | **85.25%** | **84.99%** | Автоматический подбор примеров дал прирост **+4%** к ручному методу и **+10%** к BERT. |
| **Manual Prompt** | Gemma-3-27B | 81.25% | 82.56% | Ручной инжиниринг эффективен, но требует времени. |
| **GEPA** | Gemma-3-27B (DSPy) | 76.12% | 79.31% | Генетическая оптимизация ушла в переусложнение инструкций. |
| **Fine-Tuning** | ModernBERT-large | 75.88% | 73.60% | Базовый результат, encoder-модели уступают LLM на малых данных. |

**Вывод:**
Использование больших LLM с автоматической оптимизацией промптов на задаче детекции сарказма в условиях ограниченного количества размеченных данных может показывать значительно лучшие результаты, чем классический Fine-Tuning BERT, но при этом данный подход стабильно тратит больше времени на инференс по сравнению с классическими методами.